# Libraries

In [1]:
import numpy as np
import cv2
import os

# Load data

In [2]:
train_path='C:/Users/lsreeram/Downloads/_/Step01_Learning/04_ComputerVision/01_POC_Hexagon_AI_PID/03_train'
os.chdir(train_path)

In [3]:
# Load the image and symbol templates
image = cv2.imread('5258-601-A1-0414.jpeg', cv2.IMREAD_COLOR)

In [4]:
template_list = ['template (1).png',
'template (2).png',
'template (3).png',
'template (4).png',
'template (5).png',
'template (6).png']
# Add more template filenames as needed

In [5]:
template = 'template (2).png'

In [6]:
color_green =(0, 255, 0)
color_pink = (255, 105, 180)
color_deep_pink = (255, 20, 147)# Pink color (RBG format)

In [7]:
sift = cv2.SIFT_create()

In [8]:
# Define a list to store the matched templates & locations
matched_locations = []
matched_templates = []

In [9]:
# Define a threshold for matching score
threshold = 0.66

# NMS

In [10]:
# Iterate over the templates
for template_filename in template_list:
    # Read the template image
    template = cv2.imread(template_filename, cv2.IMREAD_COLOR)

    # Convert the template and image to grayscale
    gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    result = cv2.matchTemplate(gray_image, gray_template, cv2.TM_CCOEFF_NORMED)

    # Find locations where the matching score exceeds the threshold
    locations = np.where(result >= threshold)

    # Iterate over the locations and store the matched templates
    for loc in zip(*locations[::-1]):
        matched_templates.append(template_filename)
        #print(f"Template '{template_filename}' found at coordinates (x, y): {loc}")
        x, y = loc
        w, h = template.shape[1], template.shape[0]
        matched_locations.append((x, y, x + w, y + h))
        #print("Template found at coordinates (x, y):", x, y)

# NMS with overlap

In [ ]:
def non_maximum_suppression(image, template, threshold):
    # Convert the image and template to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    result = cv2.matchTemplate(gray_image, gray_template, cv2.TM_CCOEFF_NORMED)
    _, _, _, max_loc = cv2.minMaxLoc(result)

    # Bounding box coordinates for the detected region
    x1 = max_loc[0]
    y1 = max_loc[1]
    x2 = x1 + template.shape[1]
    y2 = y1 + template.shape[0]
    
    # Calculate the overlap area of the selected bounding box with the remaining bounding boxes
    selected_box_area = (x2 - x1 + 1) * (y2 - y1 + 1)
    remaining_boxes_area = (result.shape[1] - template.shape[1] + 1) * \
                           (result.shape[0] - template.shape[0] + 1)
    
    overlap_area = np.maximum(0, x2 - x1 + 1) * np.maximum(0, y2 - y1 + 1)
    
    # Calculate the overlap ratio
    overlap_ratio = overlap_area / (selected_box_area + remaining_boxes_area - overlap_area)
    
    if overlap_ratio >= threshold:
        return [(x1, y1, x2, y2)]
    else:
        return []

In [ ]:
# Iterate over the templates
for template_filename in template_list:
    # Read the template image
    template = cv2.imread(template_filename, cv2.IMREAD_COLOR)

    # Perform non-maximum suppression on the matched locations
    locations = non_maximum_suppression(image, template, threshold)

    # Iterate over the locations and store the matched templates
    for loc in locations:
        matched_templates.append(template_filename)
        matched_locations.append(loc)

# NMS with Template Matching Technique/SIFT

In [ ]:
# Iterate over the templates
for template_filename in template_list:
    # Read the template image
    template = cv2.imread(template_filename, cv2.IMREAD_COLOR)

    # Convert the template and image to grayscale
    gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Perform SIFT feature extraction on the template and image
    keypoints_template, descriptors_template = sift.detectAndCompute(gray_template, None)
    keypoints_image, descriptors_image = sift.detectAndCompute(gray_image, None)

    # Use FLANN for feature matching
    matcher = cv2.FlannBasedMatcher_create()
    matches = matcher.knnMatch(descriptors_template, descriptors_image, k=2)

    # Apply ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    # Extract the coordinates of the matched keypoints
    for match in good_matches:
        x, y = keypoints_image[match.trainIdx].pt
        w, h = template.shape[1], template.shape[0]
        matched_templates.append(template_filename)
        matched_locations.append((int(x), int(y), int(x + w), int(y + h)))
                # Extract the keypoints for the good matches
        points_template = np.float32([keypoints_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        points_image = np.float32([keypoints_image[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

In [ ]:
# Ensure that there are at least 4 corresponding points
if len(points_template) >= 4 and len(points_image) >= 4:
    # Find the homography matrix
    M, mask = cv2.findHomography(points_template, points_image, cv2.RANSAC, 5.0)

    # Apply perspective transformation to the template corners
    h, w = template.shape[:2]
    template_corners = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
    transformed_corners = cv2.perspectiveTransform(template_corners, M)

    # Draw the bounding box on the input image
    image_with_box = cv2.polylines(image, [np.int32(transformed_corners)], True, (0, 255, 0), 2)
    # Get the screen resolution
    screen_width, screen_height = 1920, 1080  # Replace with your screen resolution
    
    # Resize the image to the desired resolution
    image_new = cv2.resize(image_with_box, (screen_width, screen_height))

    # Display the result
    cv2.imshow('Image with Bounding Box', image_new)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print('Insufficient corresponding points to calculate homography.')


# Draw Shapes

In [11]:
# Draw bounding boxes around the matched locations on the image
for (x1, y1, x2, y2) in matched_locations:
    cv2.rectangle(image, (x1,y1), (x2, y2), color_deep_pink,2)

# Show Output

In [12]:
# Get the screen resolution
screen_width, screen_height = 1920, 1080  # Replace with your screen resolution

# Resize the image to the desired resolution
image_output = cv2.resize(image, (screen_width, screen_height))

# Create a named window
cv2.namedWindow('Image', cv2.WINDOW_NORMAL)

# Display the image
cv2.imshow('Image', image_output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Define a list to store the matched templates & locations
matched_locations = []
matched_templates = []